Imports and Utility


In [ ]:
pip install -qU numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv pdfplumber

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase, cosine_similarity
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Documents

## Loading Source Documents

In [2]:
text_loader = TextFileLoader("data/machine-learning-yearning.pdf")
documents = text_loader.load_documents()
len(documents)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>> data/machine-learning-yearning.pdf


1

In [ ]:
print(documents[0][:100])

## Splitting Text Into Chunks

In [3]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

193

In [4]:
split_documents[0:1]

['  Machine Learning Yearning is a\ndeeplearning.ai project.\n© 2018 Andrew Ng. All Rights Reserved.\nPage 2 Machine Learning Yearning-Draft Andrew Ng Table of Contents\n1 Why Machine Learning Strategy\n2 How to use this book to help your team\n3 Prerequisites and Notation\n4 Scale drives machine learning progress\n5 Your development and test sets\n6 Your dev and test sets should come from the same distribution\n7 How large do the dev/test sets need to be?\n8 Establish a single-number evaluation metric for your team to optimize\n9 Optimizing and satisficing metrics\n10 Having a dev set and metric speeds up iterations\n11 When to change dev/test sets and metrics\n12 Takeaways: Setting up development and test sets\n13 Build your first system quickly, then iterate\n14 Error analysis: Look at dev set examples to evaluate ideas\n15 Evaluating multiple ideas in parallel during error analysis\n16 Cleaning up mislabeled dev and test set examples\n17 If you have a large dev set, split it into t

# Task 3: Embeddings and Vectors

In [5]:
from aimakerspace.openai_utils.embedding import EmbeddingModel
embedding_model = EmbeddingModel()
puppy_sentence = "I love puppies!"
dog_sentence = "I love dogs!"
puppy_vector = embedding_model.get_embedding(puppy_sentence)
dog_vector = embedding_model.get_embedding(dog_sentence)
cosine_similarity(puppy_vector, dog_vector)

0.8339742745295414

In [6]:
puppy_sentence = "I love puppies!"
cat_sentence = "I dislike cats!"
puppy_vector = embedding_model.get_embedding(puppy_sentence)
cat_vector = embedding_model.get_embedding(cat_sentence)

cosine_similarity(puppy_vector, cat_vector)

0.3725204049494281

# Vector Database

## Question #1:
The default embedding dimension of text-embedding-3-small is 1536, as noted above.

1. Is there any way to modify this dimension? Yes pass `dimensions` parameter with the required value to modify default embedding dimension
2. What technique does OpenAI use to achieve this? OpenAI does it through `Matryoshka Representation Learning`

In [7]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

## Question #2:
What are the benefits of using an async approach to collecting our embeddings?

Benefits of using an async approach is performance. Performance can be improved for a code that is IO bound, when the system is waiting for a response for one request, it can process something else in a meanwhile (prepare and send all other request in our case). 

In [8]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('nts:\n1. Parser: A system that annotates the text with information identifying the most\n15\nimportant words. For example, you might use the parser to label all the adjectives\nand nouns. You would therefore get the following annotated text:\nThis is a great mop !\n\u200bAdjective \u200bNoun\n\u200b \u200b\n2. Sentiment classifier: A learning algorithm that takes as input the annotated text and\npredicts the overall sentiment. The parser’s annotation could help this learning\nalgorithm greatly: By giving adjectives a higher weight, your algorithm will be able to\nquickly hone in on the important words such as “great,” and ignore less important\nwords such as “this.”\nWe can visualize your “pipeline” of two components as follows:\nThere has been a recent trend toward replacing pipeline systems with a single learning\nalgorithm. An end-to-end learning algorithm for this task would simply take as input\n\u200b \u200b\nthe raw, original text “This is a great mop!”, and try to directly r

# ChatOpenAI
## Question #3:
When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?
Yes, set the seed parameter to any integer of your choice and use the same value across requests you'd like deterministic outputs for.
Ensure all other parameters (like prompt or temperature) are the exact same across requests.

# Creating and Prompting OpenAI's gpt-3.5-turbo!

In [9]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]
response = chat_openai.run(messages)

In [10]:
print(response)

The best way to write a loop ultimately depends on the specific requirements of your program and personal preference. However, in general, using the most appropriate looping construct for the task at hand is key. 

For simple iteration over a range of values, a `for` loop is often the easiest and most readable choice. For example:

```python
for i in range(5):
    print(i)
```

If you need to iterate based on a condition, a `while` loop might be more appropriate:

```python
n = 0
while n < 5:
    print(n)
    n += 1
```

Remember to ensure your loop has a proper exit condition to prevent infinite loops. Additionally, using clear and descriptive variable names, as well as commenting your code, can help improve readability and maintainability.


# Task 5: Retrieval Augmented Generation

In [11]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""

user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_user_prompt, formatted_system_prompt]), "context" : context_list}

## Question #4:
What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?
It is called `Chain of Thought Prompting`

In [12]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [13]:
retrieval_augmented_qa_pipeline.run_pipeline("What is the 'Michael Eisner Memorial Weak Executive Problem'?")


{'response': "I don't know.",
 'context': [('nts:\n1. Parser: A system that annotates the text with information identifying the most\n15\nimportant words. For example, you might use the parser to label all the adjectives\nand nouns. You would therefore get the following annotated text:\nThis is a great mop !\n\u200bAdjective \u200bNoun\n\u200b \u200b\n2. Sentiment classifier: A learning algorithm that takes as input the annotated text and\npredicts the overall sentiment. The parser’s annotation could help this learning\nalgorithm greatly: By giving adjectives a higher weight, your algorithm will be able to\nquickly hone in on the important words such as “great,” and ignore less important\nwords such as “this.”\nWe can visualize your “pipeline” of two components as follows:\nThere has been a recent trend toward replacing pipeline systems with a single learning\nalgorithm. An end-to-end learning algorithm for this task would simply take as input\n\u200b \u200b\nthe raw, original text “Th

In [14]:
retrieval_augmented_qa_pipeline.run_pipeline("How to clean up mislabeled dev and test set examples?")

{'response': "I don't see specific information in the provided text regarding how to clean up mislabeled dev and test set examples.",
 'context': [(', then perhaps the reward is R(T) = -1,000—a huge negative reward. A\n\u200b \u200b\ntrajectory T resulting in a safe landing might result in a positive R(T) with the exact value\n\u200b \u200b \u200b \u200b\ndepending on how smooth the landing was. The reward function R(.) is typically chosen by\n\u200b \u200b\nhand to quantify how desirable different trajectories T are. It has to trade off how bumpy the\n\u200b \u200b\nlanding was, whether the helicopter landed in exactly the desired spot, how rough the ride\ndown was for passengers, and so on. It is not easy to design good reward functions.\nPage 88 Machine Learning Yearning-Draft Andrew Ng Given a reward function R(T), the job of the reinforcement learning algorithm is to control\n\u200b \u200b\nthe helicopter so that it achieves max R(T). However, reinforcement learning algorithms\n\u

# Activity #1:
Enhance your RAG application in some way!
- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

# Visibility Tooling

In [ ]:
pip install -qU wandb

In [15]:
import getpass
import os
wandb_key = getpass.getpass("Weights and Biases API Key: ")
os.environ["WANDB_API_KEY"] = wandb_key

In [16]:
import wandb

wandb.init(project="Visibility Example - AIE3")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: syedwaseemjan (test11111111111). Use `wandb login --relogin` to force relogin


In [17]:
import datetime
from wandb.sdk.data_types.trace_tree import Trace

class RetrievalAugmentedGenerationPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, wandb_project = None) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.wandb_project = wandb_project

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)


        start_time = datetime.datetime.now().timestamp() * 1000

        try:
            openai_response = self.llm.run([formatted_system_prompt, formatted_user_prompt], text_only=False)
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "success"
            status_message = (None, )
            response_text = openai_response.choices[0].message.content
            token_usage = dict(openai_response.usage)
            model = openai_response.model

        except Exception as e:
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "error"
            status_message = str(e)
            response_text = ""
            token_usage = {}
            model = ""

        if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage,
                    "model_name" : model
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
        
        return {"response" : response_text, "context" : context_list} if response_text else "We ran into an error. Please try again later. Full Error Message: " + status_message


In [18]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedGenerationPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    wandb_project="LLM Visibility Example"
)

In [19]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?")


{'response': "I don't know.",
 'context': [('ls.\nNow suppose your training set has 100 examples. Perhaps even a few examples are\nmislabeled, or ambiguous—some images are very blurry, so even humans cannot tell if there\nis a cat. Perhaps the learning algorithm can still “memorize” most or all of the training set,\nbut it is now harder to obtain 100% accuracy. By increasing the training set from 2 to 100\nexamples, you will find that the training set accuracy will drop slightly.\nFinally, suppose your training set has 10,000 examples. In this case, it becomes even harder\nfor the algorithm to perfectly fit all 10,000 examples, especially if some are ambiguous or\nmislabeled. Thus, your learning algorithm will do even worse on this training set.\nLet’s add a plot of training error to our earlier figures:\nYou can see that the blue “training error” curve increases with the size of the training set.\nFurthermore, your algorithm usually does better on the training set than on the dev set;

In [20]:
retrieval_augmented_qa_pipeline.run_pipeline("What are some tips for being an effective CEO?")


{'response': "I don't know.",
 'context': [('ata that came from the same distribution as the dev/test set (mobile images).\nPage 76 Machine Learning Yearning-Draft Andrew Ng 40 Generalizing from the training set to the\ndev set\nSuppose you are applying ML in a setting where the training and the dev/test distributions\nare different. Say, the training set contains Internet images + Mobile images, and the\ndev/test sets contain only Mobile images. However, the algorithm is not working well: It has\na much higher dev/test set error than you would like. Here are some possibilities of what\nmight be wrong:\n1. It does not do well on the training set. This is the problem of high (avoidable) bias on the\ntraining set distribution.\n2. It does well on the training set, but does not generalize well to previously unseen data\ndrawn from the same distribution as the training set. This is high variance.\n\u200b\n3. It generalizes well to new data drawn from the same distribution as the training s

## Question #5:
What is the model_name from the WandB root_span trace?
gpt-3.5-turbo-0125

# RAG Evaluation Using GPT-4

In [21]:
query = "What are some tips for making an effective model?"

response = retrieval_augmented_qa_pipeline.run_pipeline(query)

print(response["response"])

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response and the provided context
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Context: {context}
Response: {response}"""

try:
    chat_openai = ChatOpenAI(model_name="gpt-4-turbo")
except:
    chat_openai = ChatOpenAI()

evaluator_system_prompt = SystemRolePrompt(evaluator_system_template)
evaluation_prompt = UserRolePrompt(evaluation_template)

messages = [
    evaluator_system_prompt.create_message(format=False),
    evaluation_prompt.create_message(
        input=query,
        context="\n".join([context[0] for context in response["context"]]),
        response=response["response"]
    ),
]

chat_openai.run(messages, response_format={"type" : "json_object"})

I don't know


'{"clarity" : "2", "faithfulness" : "1", "correctness" : "0"}'

In [ ]:
wandb.finish()